# Transformer

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import string
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pickle
import pandas as pd


from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.activations import relu
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    Embedding,
    Flatten,
    Conv1D,
    MaxPooling1D,
    Add,
    Lambda,
    Dropout,
    concatenate,
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.backend import l2_normalize
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from IPython.display import SVG

In [3]:
def ModiData(obj_df):
    obj_df = obj_df[obj_df["label"] != 2]
    # obj_df["label"] = obj_df["label"].astype(string)
    obj_df.label = obj_df.label.replace(0, "neg")
    obj_df.label = obj_df.label.replace(1, "neg")
    obj_df.label = obj_df.label.replace(3, "pos")
    obj_df.label = obj_df.label.replace(4, "pos")
    obj_df = obj_df.fillna(0)
    # obj_df["label"] = obj_df["label"].astype(int)
    return obj_df


def clean_line(line):
    # split into tokens by white space
    tokens = line.split()
    # remove punctuation from each token
    table = str.maketrans("", "", string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words("english"))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens


def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer


def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding="post")
    return padded

In [4]:
train_df = pd.read_csv(
    "/content/drive/MyDrive/Sentiment Analysis/IMDB/train.csv", 
)

test_df = pd.read_csv(
    "/content/drive/MyDrive/Sentiment Analysis/IMDB/test.csv", 
)

train_df = train_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [5]:
# train_df = ModiData(train_df)
# # dev_df = ModiData(dev_df)
# test_df = ModiData(test_df)

train_x = train_df.text.tolist()
train_y = train_df.label
# dev_x = dev_df.text.tolist()
# dev_y = dev_df.label
test_x = test_df.text.tolist()
test_y = test_df.label

max_len = 200
# tokenizer = create_tokenizer(train_x)
with open("/content/drive/MyDrive/Sentiment Analysis/tokenizer/200in_IMDB.pickle", "rb") as handle:
    tokenizer = pickle.load(handle)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
word_index = tokenizer.word_index
train_x = encode_text(tokenizer, train_x, max_len)
# dev_x = encode_text(tokenizer, dev_x, max_len)
test_x = encode_text(tokenizer, test_x, max_len)

le = LabelEncoder()
train_y = le.fit_transform(train_y).reshape(-1, 1)
# dev_y = le.transform(dev_y).reshape(-1, 1)
test_y = le.transform(test_y).reshape(-1, 1)

## Model

In [6]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim

        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim,
        })
        return config

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.vocab_size = vocab_size
        self.maxlen = maxlen
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'vocab_size': self.vocab_size,
            'maxlen': self.maxlen,
            'embed_dim': self.embed_dim,
        })
        return config

In [7]:
def get_Trans(max_len):
    embed_dim = 50  # Embedding size for each token
    num_heads = 2  # Number of attention heads
    ff_dim = 32  # Hidden layer size in feed forward network inside transformer


    embedding_layer = TokenAndPositionEmbedding(max_len, len(word_index)+1, embed_dim)

    inputs = Input(shape=(max_len,), dtype="int32", name="input")
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(20, activation="relu")(x)
    # x = layers.Dropout(0.1)(x)

    o = Dense(1, activation="sigmoid", name="output")(x)

    model = Model(inputs=inputs, outputs=o)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])

    return model

In [11]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("val_loss") < 0.3:
            print("\nReached 0.3 loss so cancelling training!")
            self.model.stop_training = True


beststop = myCallback()
earlystop = EarlyStopping(monitor="val_loss", min_delta=0.0001)
checkpointer = ModelCheckpoint(filepath="./weights.hdf5", verbose=1)

In [12]:
def TrainModel(num):
    while num > 0:
        model = get_Trans(max_len)
        # model.summary()
        history = model.fit(
            x=train_x,
            y=train_y,
            batch_size=50,
            epochs=20,
            validation_data=(test_x, test_y),
            callbacks=[earlystop, beststop],
        )
        if history.history["val_acc"][-1] >= 0.875:
            save_name = (
                "/content/drive/MyDrive/Sentiment Analysis/Model/Trans_IMDB/"
                + str(history.history["val_loss"][-1])[2:6]
                + "_"
                + str(history.history["val_acc"][-1])[2:6]
                + ".hdf5"
            )
            num -= 1
            model.save(save_name)

In [13]:
TrainModel(9)

Epoch 1/20
500/500 [==============================] - 31s 59ms/step - loss: 0.4934 - acc: 0.7308 - val_loss: 0.3619 - val_acc: 0.8390
Epoch 2/20
500/500 [==============================] - 29s 58ms/step - loss: 0.1429 - acc: 0.9495 - val_loss: 0.3787 - val_acc: 0.8450
Epoch 1/20
500/500 [==============================] - 31s 60ms/step - loss: 0.4837 - acc: 0.7398 - val_loss: 0.2949 - val_acc: 0.8733

Reached 0.3 loss so cancelling training!
Epoch 1/20
500/500 [==============================] - 31s 59ms/step - loss: 0.5214 - acc: 0.7017 - val_loss: 0.2900 - val_acc: 0.8779

Reached 0.3 loss so cancelling training!
Epoch 1/20
500/500 [==============================] - 32s 60ms/step - loss: 0.5250 - acc: 0.6915 - val_loss: 0.3117 - val_acc: 0.8683
Epoch 2/20
500/500 [==============================] - 30s 60ms/step - loss: 0.1530 - acc: 0.9446 - val_loss: 0.3268 - val_acc: 0.8666
Epoch 1/20
500/500 [==============================] - 31s 60ms/step - loss: 0.4631 - acc: 0.7668 - val_loss: 0.3

In [ ]:
# del model
model = get_Trans(max_len)
model.summary()


Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 200)]             0         
_________________________________________________________________
token_and_position_embedding (None, 200, 100)          1403600   
_________________________________________________________________
transformer_block_11 (Transf (None, 200, 100)          174664    
_________________________________________________________________
global_average_pooling1d_11  (None, 100)               0         
_________________________________________________________________
dropout_46 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 20)                2020      
_________________________________________________________________
dropout_47 (Dropout)         (None, 20)                0  

In [ ]:
history = model.fit(
            x=train_x,
            y=train_y,
            batch_size=100,
            epochs=20,
            validation_data=(test_x, test_y),
            # callbacks=[earlystop],
        )

Epoch 1/20
250/250 [==============================] - 10s 35ms/step - loss: 0.5592 - acc: 0.6759 - val_loss: 0.2879 - val_acc: 0.8778
Epoch 2/20
250/250 [==============================] - 9s 35ms/step - loss: 0.2289 - acc: 0.9105 - val_loss: 0.3187 - val_acc: 0.8707
Epoch 3/20
115/250 [============>.................] - ETA: 3s - loss: 0.1673 - acc: 0.9393

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.models import load_model

# 保存模型
# model.save("10in_32unit_temp_res.h5")
# del model  # deletes the existing model
# 导入已经训练好的模型
# model = load_model("my_model.h5")
## 保存训练好的Tokenizer，和导入
import pickle

# saving
with open("/content/drive/MyDrive/Sentiment Analysis/tokenizer/49in_SST.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
loss, accuracy = model.evaluate(x=[test_x, test_x], y=test_y, verbose=0)
print(loss)
print(accuracy)

ValueError: ignored

### google

In [ ]:
def load_google():
    Word2VecModel = gensim.models.KeyedVectors.load_word2vec_format(
        r"E:\CS\MLT\GoogleNews-vectors-negative300.bin", binary=True
    )

    vocab_list = [word for word, Vocab in Word2VecModel.wv.vocab.items()]  # 存储 所有的 词语

    # word_index = {" ": 0}  # 初始化 `[word : token]` ，后期 tokenize 语料库就是用该词典。
    embeddings_index = {}  # 初始化`[word : vector]`字典

    for i in range(len(vocab_list)):
        # print(i)
        word = vocab_list[i]  # 每个词语
        # word_index[word] = i + 1 # 词语：序号
        embeddings_index[word] = Word2VecModel.wv[word]  # 词语：词向量
        # embeddings_matrix[i + 1] = Word2VecModel.wv[word]  # 词向量矩阵
    return embeddings_index


def load_fasttext_embeddings():
    glove_dir = r"E:\CS\MLT\glove.6B"
    embeddings_index = {}
    f = open(os.path.join(glove_dir, "glove.6B.50d.txt"), encoding="utf-8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs
    f.close()
    print("Found %s word vectors." % len(embeddings_index))
    return embeddings_index


def create_embeddings_matrix(embeddings_index, vocabulary, embedding_dim=100):
    embeddings_matrix = np.random.rand(len(vocabulary) + 1, embedding_dim)
    for i, word in enumerate(vocabulary):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector
    print("Matrix shape: {}".format(embeddings_matrix.shape))
    return embeddings_matrix


embeddings_index = load_google()
embeddings_matrix = create_embeddings_matrix(embeddings_index, word_index, 300)

In [ ]:
import gensim

Word2VecModel = gensim.models.KeyedVectors.load_word2vec_format(
    r"E:\CS\MLT\GoogleNews-vectors-negative300.bin", binary=True
)

vocab_list = [word for word, Vocab in Word2VecModel.wv.vocab.items()]  # 存储 所有的 词语

word_index = {" ": 0}  # 初始化 `[word : token]` ，后期 tokenize 语料库就是用该词典。
word_vector = {}  # 初始化`[word : vector]`字典

for i in range(len(vocab_list)):
    # print(i)
    word = vocab_list[i]  # 每个词语
    #     word_index[word] = i + 1 # 词语：序号
    #     word_vector[word] = Word2VecModel.wv[word] # 词语：词向量
    embeddings_matrix[i + 1] = Word2VecModel.wv[word]  # 词向量矩阵